In [ ]:
import torch
import os 
import random
import cv2 
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler, StableDiffusionUpscalePipeline

# Draw random input sample

In [ ]:
# Set the parameters:
DEVICE = "cuda"
# ... image loadiadng code ...
folder_path = "../../public_data/DOG/images"

# Check if the folder exists
if os.path.exists(folder_path):
    # List all files in the folder
    files = os.listdir(folder_path)

filepath = f"{folder_path}/{random.choice(files)}"

image = cv2.imread(filepath)

plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title('Random Image')
plt.show()


In [ ]:
image.shape, type(image)

In [ ]:
image

In [ ]:
filepath

# Upscale Input Image

In [ ]:
# load model and scheduler
model_id = "stabilityai/stable-diffusion-x4-upscaler"
upscale_pipeline = StableDiffusionUpscalePipeline.from_pretrained(
    model_id, revision="fp16", torch_dtype=torch.float16
)
upscale_pipeline = upscale_pipeline.to("cuda")

upscale_pipeline.enable_model_cpu_offload()
# remove following line if xFormers is not installed or you have PyTorch 2.0 or higher installed
upscale_pipeline.enable_xformers_memory_efficient_attention()


In [ ]:
upscaled_image = upscale_pipeline(prompt="", image=Image.fromarray(image)).images[0]
upscaled_image

# Edge dection on the upscaled image

In [ ]:
upscaled_image = np.array(upscaled_image)

low_threshold = 100
high_threshold = 200

canny_image = cv2.Canny(upscaled_image, low_threshold, high_threshold)
canny_image = canny_image[:, :, None]
canny_image = np.concatenate([canny_image, canny_image, canny_image], axis=2)
canny_image = Image.fromarray(canny_image)


In [ ]:
canny_image

In [ ]:
# Load ControlNet and SD1.5 model
controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16, safety_checker =None,
    requires_safety_checker = False
).to(DEVICE)

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

pipe.enable_model_cpu_offload()
# remove following line if xFormers is not installed or you have PyTorch 2.0 or higher installed
pipe.enable_xformers_memory_efficient_attention()


In [ ]:
PROMPT = "dog"
NEGATIVE_PROMPT = "Amputee, Autograph, Bad anatomy, Bad illustration, Bad proportions, Beyond the borders, Blank background, Blurry, Body out of frame, Boring background, Branding, Cropped, Cut off, Deformed, Disfigured, Dismembered, Disproportioned, Distorted, Draft, Duplicate, Duplicated features, Extra arms, Extra fingers, Extra hands, Extra legs, Extra limbs, Fault, Flaw, Fused fingers, Grains, Grainy, Gross proportions, Hazy, Identifying mark, Improper scale, Incorrect physiology, Incorrect ratio, Indistinct, Kitsch, Logo, Long neck, Low quality, Low resolution, Macabre, Malformed, Mark, Misshapen, Missing arms, Missing fingers, Missing hands, Missing legs, Mistake, Morbid, Mutated hands, Mutation, Mutilated, Off-screen, Out of frame, Outside the picture, Pixelated, Poorly drawn face, Poorly drawn feet, Poorly drawn hands, Printed words, Render, Repellent, Replicate, Reproduce, Revolting dimensions, Script, Shortened, Sign, Signature, Split image, Squint, Storyboard, Text, Tiling, Trimmed, Ugly, Unfocused, Unattractive, Unnatural pose, Unreal engine, Unsightly, Watermark, Written language"

# Run image generation
out_images = pipe(
    prompt=PROMPT,
    negative_prompt=NEGATIVE_PROMPT,
    image=canny_image,
    height=512,
    width=512,
    num_images_per_prompt=1,
    #guidance_scale=1,
    num_inference_steps=50)

out_images[0][0]

In [ ]:
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title('Random Image')
plt.show()

In [ ]:
out_images[0][0].resize((128, 128))
